In [ ]:
!git clone https://github.com/vladislavkorenik/image-restoration
!mv image-restoration/* ./

In [112]:
import cv2
import math
import torch
import numpy as np
from noise import NoisyDataset
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torch.utils.data import DataLoader

def show(img, a):

    plt.figure()
    if a == 'v':
        npimg = torch.squeeze(img).data.cpu().numpy()
    else:
        npimg = torch.squeeze(img).numpy()

    transposeImg = np.transpose(npimg, (1,2,0))
    plt.imshow(transposeImg)

    return transposeImg

def ssim(img1, img2):

    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calculate_ssim(img1, img2):

    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for itr in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

def calculate_mse(img1, img2):

    img1 = img1
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)

    return mse

def calculate_psnr(img1, img2):
    
    mse = calculate_mse(img1, img2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))

In [ ]:
modelName = 'multiplicative_bernoulli'
noiseParam = 0.85

# modelName = 'text'
# noiseParam = 0.3

# modelName = 'gaussian'
# noiseParam = 60

In [ ]:
from model import NetModel
from train_utils import Train

architecture = NetModel(3, 64)

params = {
    'noise_model': (modelName, noiseParam),
    'crop_size': 64,
    'clean_targs': False,
    'lr': 0.001,
    'epochs': 100,
    'bs': 2,
    'cuda': True
}

trainer = Train(architecture, 'dataset/train', 'dataset/valid', params)
trainer.train()
model = trainer.architecture

torch.save(model,modelName + '.pt')

In [109]:
model = torch.load('trained/' + modelName + '.pt')

In [ ]:
data = NoisyDataset('dataset/valid/', crop_size=128, train_noise_model=(modelName, noiseParam), clean_targ=True)
loadedData = DataLoader(data, batch_size=1, shuffle=True)

i = 0
for _list in loadedData:
    plt.figure()
    pred = model(Variable(_list[0].cuda()))
    pred_img = show(pred, 'v')
    noise_img = show(_list[0], 't')
    source_img = show(_list[-1], 't')

    if i == 0:
        break

psnr_val = calculate_psnr(noise_img, pred_img)
ssim_val = calculate_ssim(source_img, pred_img)
mse_val = calculate_mse(source_img, pred_img)
print('PSNR: ' + str(psnr_val) + ' SSIM: ' + str(ssim_val) + ' MSE: ' + str(mse_val))